In [27]:
import sqlite3
import datetime

# Conectar ao banco de dados
conn = sqlite3.connect("usuarios.db")
cursor = conn.cursor()

# Criar tabela de usuários se não existir
cursor.execute("""
    CREATE TABLE IF NOT EXISTS usuarios (
        id INTEGER PRIMARY KEY,
        nome TEXT NOT NULL,
        saldo REAL NOT NULL DEFAULT 0.0
    );
""")

# Criar tabela de itens se não existir
cursor.execute("""
    CREATE TABLE IF NOT EXISTS itens (
        id INTEGER PRIMARY KEY,
        usuario_id INTEGER,
        item TEXT NOT NULL,
        FOREIGN KEY (usuario_id) REFERENCES usuarios (id)
    );
""")

# Criar tabela de transações se não existir
cursor.execute("""
    CREATE TABLE IF NOT EXISTS transacoes (
        id INTEGER PRIMARY KEY,
        usuario_origem_id INTEGER,
        usuario_destino_id INTEGER,
        item TEXT,
        valor REAL,
        data DATE DEFAULT CURRENT_DATE,
        FOREIGN KEY (usuario_origem_id) REFERENCES usuarios (id),
        FOREIGN KEY (usuario_destino_id) REFERENCES usuarios (id)
    );
""")

# Função para cadastrar usuário
def cadastrar_usuario(nome, saldo=0.0):
    cursor.execute("INSERT INTO usuarios (nome, saldo) VALUES (?,?)", (nome, saldo))
    conn.commit()
    print(f"Usuário {nome} cadastrado com sucesso!")

# Função para adicionar item ao usuário
def adicionar_item(usuario_id, item):
    cursor.execute("INSERT INTO itens (usuario_id, item) VALUES (?,?)", (usuario_id, item))
    conn.commit()
    print(f"Item '{item}' adicionado ao usuário com sucesso!")

# Função para consultar saldo de usuário
def consultar_saldo(nome):
    cursor.execute("SELECT saldo FROM usuarios WHERE nome =?", (nome,))
    saldo = cursor.fetchone()
    if saldo:
        print(f"Saldo de {nome}: R$ {saldo[0]:.2f}")
    else:
        print(f"Usuário {nome} não encontrado.")

# Função para atualizar saldo de usuário
def atualizar_saldo(nome, saldo):
    cursor.execute("UPDATE usuarios SET saldo =? WHERE nome =?", (saldo, nome))
    conn.commit()
    print(f"Saldo de {nome} atualizado para R$ {saldo:.2f}")

# Função para consultar itens do usuário
def consultar_itens(nome):
    cursor.execute("SELECT i.item FROM usuarios u JOIN itens i ON u.id = i.usuario_id WHERE u.nome =?", (nome,))
    itens = cursor.fetchall()
    if itens:
        print(f"Itens do usuário {nome}:")
        for item in itens:
            print(f"- {item[0]}")
    else:
        print(f"Usuário {nome} não encontrado.")

# Função para mudar usuário
def mudar_usuario():
    global nome
    nome = input("Digite o nome do novo usuário: ")
    cursor.execute("SELECT saldo FROM usuarios WHERE nome =?", (nome,))
    saldo = cursor.fetchone()
    if saldo:
        print(f"Usuário {nome} encontrado. Saldo: R$ {saldo[0]:.2f}")
    else:
        print(f"Usuário {nome} não encontrado. Criando novo usuário...")
        saldo = float(input("Digite o saldo inicial do usuário: "))
        cadastrar_usuario(nome, saldo)

# Função para realizar transação entre usuários
def realizar_transacao():
    usuario_destino = input("Digite o nome do usuário destino: ")
    cursor.execute("SELECT id FROM usuarios WHERE nome =?", (usuario_destino,))
    usuario_destino_id = cursor.fetchone()
    if usuario_destino_id:
        usuario_destino_id = usuario_destino_id[0]
        print("Opções de transação:")
        print("1. Transferir dinheiro")
        print("2. Transferir item")
        opcao_transacao = input("Digite a opção desejada: ")
        if opcao_transacao == "1":
            valor = float(input("Digite o valor a ser transferido: "))
            cursor.execute("UPDATE usuarios SET saldo = saldo -? WHERE nome =?", (valor, nome))
            cursor.execute("UPDATE usuarios SET saldo = saldo +? WHERE nome =?", (valor, usuario_destino))
            cursor.execute("INSERT INTO transacoes (usuario_origem_id, usuario_destino_id, valor) VALUES (?,?,?)", (cursor.execute("SELECT id FROM usuarios WHERE nome =?", (nome,)).fetchone()[0], usuario_destino_id, valor))
            conn.commit()
            print(f"Valor de R$ {valor:.2f} transferido para {usuario_destino} com sucesso!")
        elif opcao_transacao == "2":
            item = input("Digite o item a ser transferido: ")
            cursor.execute("SELECT i.id FROM itens i JOIN usuarios u ON i.usuario_id = u.id WHERE u.nome =? AND i.item =?", (nome, item))
            item_id = cursor.fetchone()
            if item_id:
                item_id = item_id[0]
                valor = float(input("Digite o valor a ser cobrado pelo item: "))
                cursor.execute("UPDATE usuarios SET saldo = saldo -? WHERE nome =?", (valor, usuario_destino))
                cursor.execute("UPDATE usuarios SET saldo = saldo +? WHERE nome =?", (valor, nome))
                cursor.execute("UPDATE itens SET usuario_id =? WHERE id =?", (usuario_destino_id, item_id))
                cursor.execute("INSERT INTO transacoes (usuario_origem_id, usuario_destino_id, item, valor) VALUES (?,?,?,?)", (cursor.execute("SELECT id FROM usuarios WHERE nome =?", (nome,)).fetchone()[0], usuario_destino_id, item, valor))
                conn.commit()
                print(f"Item '{item}' transferido para {usuario_destino} com sucesso! Valor de R$ {valor:.2f} cobrado.")
            else:
                print(f"Item '{item}' não encontrado.")
        else:
            print("Opção inválida. Tente novamente.")
    else:
        print(f"Usuário {usuario_destino} não encontrado.")

# Função para gerar extrato de transações
def gerar_extrato():
    print("Opções de extrato de transações:")
    print("1. Todas as transações")
    print("2. Transações de um usuário específico")
    opcao_extrato = input("Digite a opção desejada: ")
    if opcao_extrato == "1":
        cursor.execute("""
            SELECT
                t.data,
                uo.nome AS usuario_origem,
                ud.nome AS usuario_destino,
                t.valor,
                CASE WHEN t.item IS NOT NULL THEN 'Item: ' || t.item ELSE 'Transferência' END
            FROM
                transacoes t
            JOIN
                usuarios uo ON t.usuario_origem_id = uo.id
            JOIN
                usuarios ud ON t.usuario_destino_id = ud.id
            ORDER BY
                t.data DESC
        """)
        transacoes = cursor.fetchall()
        if transacoes:
            data_atual = datetime.date.today().strftime("%Y-%m-%d")
            arquivo_extrato = f"extrato_{data_atual}.txt"
            with open(arquivo_extrato, "w") as f:
                f.write("Extrato de transações:\n")
                for transacao in transacoes:
                    if transacao[3] > 0:
                        f.write(f"{transacao[0]} - {transacao[1]} enviou R$ {transacao[3]:.2f} para {transacao[2]} - {transacao[4]}\n")
                    else:
                        f.write(f"{transacao[0]} - {transacao[1]} recebeu R$ {-transacao[3]:.2f} de {transacao[2]} - {transacao[4]}\n")
            print(f"Extrato gerado com sucesso! Arquivo salvo como {arquivo_extrato}")
        else:
            print("Nenhuma transação encontrada.")
    elif opcao_extrato == "2":
        usuario = input("Digite o nome do usuário: ")
        cursor.execute("SELECT id FROM usuarios WHERE nome =?", (usuario,))
        usuario_id = cursor.fetchone()
        if usuario_id:
            usuario_id = usuario_id[0]
            cursor.execute("""
                SELECT
                    t.data,
                    uo.nome AS usuario_origem,
                    ud.nome AS usuario_destino,
                    t.valor,
                    CASE WHEN t.item IS NOT NULL THEN 'Item: '| t.item ELSE 'Transferência: 'ND
                FROM
                    transacoes t
                JOIN
                    usuarios uo ON t.usuario_origem_id = uo.id
                JOIN
                    usuarios ud ON t.usuario_destino_id = ud.id
                WHERE
                    t.usuario_origem_id =? OR t.usuario_destino_id =?
                ORDER BY
                    t.data DESC
            """, (usuario_id, usuario_id))
            transacoes = cursor.fetchall()
            if transacoes:
                print(f"Extrato de transações do usuário {usuario}:")
                for transacao in transacoes:
                    if transacao[3] > 0:
                        print(f"{transacao[0]} - {transacao[1]} enviou R$ {transacao[3]:.2f} para {transacao[2]} - {transacao[4]}")
                    else:
                        print(f"{transacao[0]} - {transacao[1]} recebeu R$ {-transacao[3]:.2f} de {transacao[2]} - {transacao[4]}")
            else:
                print(f"Nenhuma transação encontrada para o usuário {usuario}.")
        else:
            print(f"Usuário {usuario} não encontrado.")
    else:
        print("Opção inválida. Tente novamente.")

# Interface de usuário
nome = input("Digite o nome do usuário: ")
cursor.execute("SELECT saldo FROM usuarios WHERE nome =?", (nome,))
saldo = cursor.fetchone()
if saldo:
    print(f"Usuário {nome} encontrado. Saldo: R$ {saldo[0]:.2f}")
else:
    print(f"Usuário {nome} não encontrado. Criando novo usuário...")
    saldo = float(input("Digite o saldo inicial do usuário: "))
    cadastrar_usuario(nome, saldo)

while True:
    print("\nOpções:")
    print("1. Adicionar item")
    print("2. Consultar saldo")
    print("3. Atualizar saldo")
    print("4. Consultar itens")
    print("5. Mudar usuário")
    print("6. Realizar transação entre usuários")
    print("7. Gerar extrato de transações")
    print("8. Sair")
    opcao = input("Digite a opção desejada: ")

    if opcao == "1":
        item = input("Digite o item a ser adicionado: ")
        cursor.execute("SELECT id FROM usuarios WHERE nome =?", (nome,))
        usuario_id = cursor.fetchone()[0]
        adicionar_item(usuario_id, item)
    elif opcao == "2":
        consultar_saldo(nome)
    elif opcao == "3":
        saldo = float(input("Digite o novo saldo do usuário: "))
        atualizar_saldo(nome, saldo)
    elif opcao == "4":
        consultar_itens(nome)
    elif opcao == "5":
        mudar_usuario()
    elif opcao == "6":
        realizar_transacao()
    elif opcao == "7":
        gerar_extrato()
    elif opcao == "8":
        break
    else:
        print("Opção inválida. Tente novamente.")

Digite o nome do usuário: Dan Arryn
Usuário Dan Arryn encontrado. Saldo: R$ 50.00

Opções:
1. Adicionar item
2. Consultar saldo
3. Atualizar saldo
4. Consultar itens
5. Mudar usuário
6. Realizar transação entre usuários
7. Gerar extrato de transações
8. Sair
Digite a opção desejada: 8


In [29]:
import sqlite3

# Conecta ao banco de dados
conn = sqlite3.connect("usuarios.db")
cursor = conn.cursor()

def remover_usuario(usuario_id):
    print(f"Removendo usuário com ID {usuario_id}...")
    cursor.execute("DELETE FROM usuarios WHERE id =?", (usuario_id,))
    cursor.execute("DELETE FROM itens WHERE usuario_id =?", (usuario_id,))
    conn.commit()
    print("Usuário removido com sucesso!")

def consultar_usuarios():
    print("Usuários cadastrados:")
    cursor.execute("SELECT * FROM usuarios")
    usuarios = cursor.fetchall()
    for usuario in usuarios:
        print(f"ID: {usuario[0]}, Nome: {usuario[1]}")

# Consulta os usuários cadastrados
consultar_usuarios()

# Pergunta ao usuário qual ID de usuário deseja remover
usuario_id = int(input("Digite o ID do usuário que deseja remover: "))

# Remove o usuário
remover_usuario(usuario_id)

# Fecha a conexão com o banco de dados
conn.close()

Usuários cadastrados:
ID: 2, Nome: Jhon Snow
Digite o ID do usuário que deseja remover: 2
Removendo usuário com ID 2...
Usuário removido com sucesso!


In [30]:
import sqlite3
import os

# Conecta ao banco de dados
conn = sqlite3.connect("usuarios.db")  # Change 'banco.db' to 'usuarios.db'
cursor = conn.cursor()

def consultar_usuarios():
    print("Usuários cadastrados:")
    cursor.execute("SELECT * FROM usuarios")
    usuarios = cursor.fetchall()
    for usuario in usuarios:
        print(f"Usuario: {usuario[1]}")
        consultar_saldo(usuario[0])
        consultar_itens(usuario[0])
        print()

def consultar_saldo(usuario_id):
    cursor.execute("SELECT saldo FROM usuarios WHERE id = ?", (usuario_id,))
    saldo = cursor.fetchone()[0]
    print(f"Saldo: R$ {saldo:.2f}")

def consultar_itens(usuario_id):
    cursor.execute("SELECT * FROM itens WHERE usuario_id = ?", (usuario_id,))
    itens = cursor.fetchall()
    print("Itens:")
    for item in itens:
        print(f"  - {item[1]}")

def gerar_relatorio_texto():
    cursor.execute("SELECT * FROM usuarios")
    usuarios = cursor.fetchall()
    with open("relatorio_usuarios.txt", "w") as f:
        f.write("Relatório de Usuários:\n")
        for usuario in usuarios:
            f.write(f"Usuario: {usuario[1]}\n")
            cursor.execute("SELECT saldo FROM usuarios WHERE id = ?", (usuario[0],))
            saldo = cursor.fetchone()[0]
            f.write(f"Saldo: R$ {saldo:.2f}\n")
            cursor.execute("SELECT * FROM itens WHERE usuario_id = ?", (usuario[0],))
            itens = cursor.fetchall()
            f.write("Itens:\n")
            for item in itens:
                f.write(f"  - {item[1]}\n")
            f.write("\n")
    print("Relatório gerado com sucesso! Arquivo salvo como relatorio_usuarios.txt")

def fazer_backup():
    cursor.execute("SELECT * FROM usuarios")
    usuarios = cursor.fetchall()
    cursor.execute("SELECT * FROM itens")
    itens = cursor.fetchall()
    with open("backup.db", "w") as f:
        f.write("Usuarios:\n")
        for usuario in usuarios:
            f.write(f"{usuario[0]},{usuario[1]},{usuario[2]}\n")
        f.write("Itens:\n")
        for item in itens:
            f.write(f"{item[0]},{item[1]},{item[2]}\n")
    print("Backup realizado com sucesso! Arquivo salvo como backup.db")

def restaurar_backup():
    if os.path.exists("backup.db"):
        with open("backup.db", "r") as f:
            lines = f.readlines()
            usuarios = []
            itens = []
            for line in lines:
                if line.startswith("Usuarios:"):
                    continue
                elif line.startswith("Itens:"):
                    continue
                elif "," in line:
                    data = line.strip().split(",")
                    if len(data) == 3:
                        # Fix: Remove the extra element from the usuario tuple
                        usuarios.append((data[1], float(data[2])))
                    elif len(data) == 3:
                        itens.append((int(data[0]), data[1], int(data[2])))
            # Delete existing data before restoring
            cursor.execute("DELETE FROM usuarios")
            cursor.execute("DELETE FROM itens")
            # Fix: Adjusted the number of placeholders to match the data
            cursor.executemany("INSERT INTO usuarios (nome, saldo) VALUES (?,?)", usuarios)
            cursor.executemany("INSERT INTO itens VALUES (?,?,?)", itens)
            conn.commit()
        print("Backup restaurado com sucesso!")
    else:
        print("Arquivo de backup não encontrado!")

while True:
    print("\nOpções:")
    print("1. Consultar usuários")
    print("2. Gerar relatório de texto")
    print("3. Fazer backup")
    print("4. Restaurar backup")
    print("5. Fechar")
    opcao = input("Digite a opção desejada: ")

    if opcao == "1":
        consultar_usuarios()
    elif opcao == "2":
        gerar_relatorio_texto()
    elif opcao == "3":
        fazer_backup()
    elif opcao == "4":
        restaurar_backup()
    elif opcao == "5":
        print("Fechando o programa...")
        break
    else:
        print("Opção inválida. Tente novamente.")

# Fecha a conexão com o banco de dados
conn.close()


Opções:
1. Consultar usuários
2. Gerar relatório de texto
3. Fazer backup
4. Restaurar backup
5. Fechar
Digite a opção desejada: 3
Backup realizado com sucesso! Arquivo salvo como backup.db

Opções:
1. Consultar usuários
2. Gerar relatório de texto
3. Fazer backup
4. Restaurar backup
5. Fechar
Digite a opção desejada: 5
Fechando o programa...
